<a href="https://colab.research.google.com/github/szymonszczurowski/Sentiment-Analysis-of-YouTube-Comments/blob/colab/Best_model_for_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selecting the best model for sentiment classification

## 1.0.0 Importing Libraries and Packages

In [14]:
!pip3 install emoji==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49719 sha256=1e602e4b4874245272acae21d6ac4509a239cbde8da89a6adead1508e1064df4
  Stored in directory: /root/.cache/pip/wheels/1b/bd/d9/310c33c45a553798a714e27e3b8395d37128425442b8c78e07
Successfully built emoji


In [15]:
!pip install transformers

In [16]:
!pip install pysentimiento

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=5a6307cef265f08388fa323034395944b181fa75f2464b9e22eeee84491575b5
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 0.6.0
    Uninstalling emoji-0.6.0:
      Successfully uninstalled emoji-0.6.0


In [3]:
import pandas as pd
import pickle

In [17]:
from transformers import pipeline, AutoTokenizer
from transformers import BertTokenizer, BertForSequenceClassification
from pysentimiento import create_analyzer


## 2.0.0 Data preparation

### 2.0.1 Loading a list of commands

In [1]:
with open('comments_list.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

print(loaded_list)


['<a href="https://www.youtube.com/watch?v=TVQgSIlN4no&amp;t=2m26s">2:26</a> duke moment', 'perfect timing on an american test and an alpha kitten bullieing a runt,....', 'Kevin is my spirit animal', 'The only antidepressant ill ever need is the office…', 'How to pick up chicks<br>1. Grow a mustache<br>2. Say &quot;Smile if you love mens prostates&quot;', 'somethin about <br>&quot;the devil you know&quot;', 'the scene with who and whom at <a href="https://www.youtube.com/watch?v=TVQgSIlN4no&amp;t=5m09s">5:09</a> was a masterpiece', 'Its always Creed 😂 he&#39;s one of my favorites', 'Michael declaring bankruptcy always kills me🤣', 'that SOUP skit is HILARIOUS as hell', 'Bro <a href="https://www.youtube.com/watch?v=TVQgSIlN4no&amp;t=1m28s">1:28</a> still makes me die to this very day lmaooo', 'It is really weak compared to UK', 'The bankruptcy part might be my favorite 😂😂😂', 'For an &quot;office moments that make me laugh lime an idiot&quot; this video is really short!', 'The “gets me ev

### 2.0.2 Label loading

In [7]:
data = pd.read_csv('comment classification.csv', header=None, names=['Komentarz', 'Klasyfikacja'])
print(data)


                                            Komentarz  Klasyfikacja
0   <a href="https://www.youtube.com/watch?v=TVQgS...             0
1   perfect timing on an american test and an alph...             0
2                           Kevin is my spirit animal             0
3   The only antidepressant ill ever need is the o...             0
4   How to pick up chicks<br>1. Grow a mustache<br...             0
..                                                ...           ...
95  Also Dwight actually rocks that pumpkin head t...             0
96  Michael hitting Meredith with the car actually...             1
97  Believe me, when its with context its even mor...             0
98  ... I dont even know where to start... I am no...            -1
99             Boy have you lost your mind!?!?!?!?!😂😂             0

[100 rows x 2 columns]


In [8]:
classification_list = data['Klasyfikacja'].tolist()
print(classification_list)


[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, -1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, -1, 0, 0, 0, 1, 1, 1, 0, 0, -1, 0, -1, 0, 0, 0, 0, -1, 0, -1, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0]


## 3.0.0 Downloading models from huggingface

### 3.0.1 ProsusAI/finbert

In [12]:
finbert = pipeline("text-classification", model = "ProsusAI/finbert")

In [22]:
finbert('I love you')

[{'label': 'neutral', 'score': 0.8632259368896484}]

### 3.0.2 lxyuan/distilbert-base-multilingual-cased-sentiments-student

In [27]:
distilled_student_sentiment_classifier = pipeline("text-classification", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student")

In [28]:
distilled_student_sentiment_classifier('I love you')

[{'label': 'positive', 'score': 0.969622790813446}]

### 3.0.3 finiteautomata/bertweet-base-sentiment-analysis

In [13]:
bertweet_base_sentiment_analysis = pipeline("text-classification", model = "finiteautomata/bertweet-base-sentiment-analysis")

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [18]:
bertweet_base_sentiment_analysis('I love you')

[{'label': 'POS', 'score': 0.9916695356369019}]

### 3.0.4 ahmedrachid/FinancialBERT-Sentiment-Analysis

In [35]:
financia_BERT_sentiment_analysis =  pipeline("text-classification", model = "ahmedrachid/FinancialBERT-Sentiment-Analysis")

In [40]:
bertweet_base_sentiment_analysis('I love you')

[{'label': 'POS', 'score': 0.9916695356369019}]